# การใช้ RAG ด้วยบริการ Azure AI Agent และ Semantic Kernel

ตัวอย่างโค้ดนี้แสดงวิธีการสร้างและจัดการ Azure AI agent สำหรับการสร้างข้อมูลแบบเสริมการดึงข้อมูล (RAG) โดยใช้ `Azure AI Agent Service` และ `Semantic Kernel` เอเจนต์จะประมวลผลคำถามของผู้ใช้โดยอ้างอิงจากบริบทที่ดึงมา และให้คำตอบที่ถูกต้องตามนั้น


การแก้ไขเวอร์ชัน SQLite  
หากคุณพบข้อผิดพลาด:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

ให้ยกเลิกการคอมเมนต์โค้ดบล็อกนี้ที่จุดเริ่มต้นของโน้ตบุ๊กของคุณ:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### การนำเข้าแพ็กเกจ  
โค้ดต่อไปนี้นำเข้าแพ็กเกจที่จำเป็น:  


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# การสร้างผลลัพธ์ด้วยการดึงข้อมูลเสริมด้วย Semantic Kernel และ Azure AI Agent Service

ตัวอย่างนี้แสดงวิธีการใช้ **Azure AI Agent Service** เพื่อดำเนินการ **การสร้างผลลัพธ์ด้วยการดึงข้อมูลเสริม (RAG)** โดยการผสมผสานโมเดลภาษากับบริบทเฉพาะทางจากเอกสารที่อัปโหลด

### วิธีการทำงาน

1. **การอัปโหลดเอกสาร**: ไฟล์ markdown (document.md) ที่มีข้อมูล (นโยบายประกันการเดินทางของ Contoso) ถูกอัปโหลดไปยังบริการตัวแทน

2. **การสร้าง Vector Store**: เอกสารถูกจัดทำดัชนีลงใน Vector Store เพื่อให้สามารถค้นหาเชิงความหมายในเนื้อหาได้

3. **การตั้งค่าตัวแทน**: ตัวแทนถูกสร้างขึ้นโดยใช้โมเดล `gpt-4o` พร้อมคำสั่งที่เข้มงวดดังนี้:
   - ตอบคำถามเฉพาะข้อมูลที่ดึงมาจากเอกสารเท่านั้น
   - ปฏิเสธการตอบคำถามหากคำถามอยู่นอกขอบเขต

4. **การรวม File Search Tool**: `FileSearchTool` ถูกลงทะเบียนกับตัวแทน เพื่อให้โมเดลสามารถค้นหาและดึงข้อมูลที่เกี่ยวข้องจากเอกสารที่จัดทำดัชนีในระหว่างการประมวลผล

5. **การโต้ตอบกับผู้ใช้**: ผู้ใช้สามารถถามคำถามได้ หากพบข้อมูลที่เกี่ยวข้องในเอกสาร ตัวแทนจะสร้างคำตอบที่มีพื้นฐานจากข้อมูลนั้น  
   หากไม่พบ ตัวแทนจะตอบอย่างชัดเจนว่าเอกสารไม่มีข้อมูลเพียงพอ


### ฟังก์ชันหลัก



ตรวจสอบให้แน่ใจว่าได้รัน `az login` โดยใช้ Azure CLI ก่อน เพื่อให้มีบริบทการตรวจสอบสิทธิ์ที่เหมาะสมในขณะที่ใช้ `DefaultAzureCredential` บริการ Azure AI Agent Service ไม่ใช้ API keys


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

แน่นอน! กรุณาให้ไฟล์ Markdown ที่ต้องการแปลมาเพื่อดำเนินการแปลให้ครับ



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลโดยอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษาจากผู้เชี่ยวชาญ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
